In [ ]:
"""commented out for test execution"""
# from google.colab import drive
# drive.mount('/content/drive')



#공용함수

In [ ]:
# 수정금지: 타임스탬프용 함수
from datetime import datetime
def printt(*args,**kwargs):
  now = datetime.now()
  now_str = "{:02}:{:02}:{:02}".format(now.hour,now.minute,now.second)
  print(now_str, *args,**kwargs)
  return int(now.hour)*60*60+int(now.minute)*60+int(now.second)

#연관 패키지 설치 (다른작업 금지)

In [ ]:
#TODO: 해당 블럭에 패키지 설치하세요.
!pip install attrdict
!pip install transformers
!pip install seqeval
!pip install fastprogress

# 파일로딩 (다른작업 금지)

In [ ]:
#TODO: 해당 블럭에 필요 파일 로딩 코드 넣으시오.


# 모델 학습

In [ ]:
# 수정금지: 타임스탬프
_model_build_start_time = printt("Model building: Start")
_model_build_start_time

In [ ]:
#TODO: 블럭에 모델 학습 - 빌딩 코드를 넣으세요. (시간측정 구간)





In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
# 실행 전 런타임 다시시작
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 
plt.plot([1, 2, 3, 4])
plt.ylabel('한국어 테스트')
plt.show()

In [ ]:
import os
!git clone https://github.com/Jaehun-Kim22/SCIC_Project.git

In [ ]:
# set config args for classification
from transformers import (
    ElectraConfig,
    ElectraTokenizer,
    ElectraForSequenceClassification,
    XLMRobertaForSequenceClassification,
    XLMRobertaTokenizer,
    XLMRobertaConfig
)
from attrdict import AttrDict
CLASSIFICATION_LIST = (
    ['None', '상담원', '상담시스템', '고객서비스', '혜택', '할부금융상품', '커뮤니티서비스',
     '카드이용/결제', '카드상품', '청구입금', '심사/한도', '생활편의서비스', '상담/채널',
     '리스렌탈상품','라이프서비스', '금융상품', '고객정보관리', '가맹점매출/승인', '가맹점대금',
     '가맹점계약', '삼성카드', '기타']
)
CLASSIFICATION_DICT = {}
for idx, item in enumerate(CLASSIFICATION_LIST):
    CLASSIFICATION_DICT[str(idx)] = item

args = AttrDict(
    {
                 'data_dir': 'SCIC_Project/assets/data',
                 'train_file': 'train_cls.txt',
                 'test_file': 'evaluation_cls.txt',
                 'task': 'sentiment',
                 'config': ElectraConfig,
                 'tokenizer': ElectraTokenizer,
                 'model': ElectraForSequenceClassification,
                #  'config': XLMRobertaConfig,
                #  'tokenizer': XLMRobertaTokenizer,
                #  'model': XLMRobertaForSequenceClassification,
                 'evaluate_test_during_training': True, 
                 'eval_all_checkpoints': True, 
                 'save_optimizer': False, 
                 'do_lower_case': False, 
                 'do_train': True, 
                 'do_eval': True, 
                 'max_seq_len': 128, 
                 'num_train_epochs': 10, 
                 'weight_decay': 0.0, 
                 'gradient_accumulation_steps': 1, 
                 'adam_epsilon': 1e-08, 
                 'warmup_proportion': 0, 
                 'max_steps': -1, 
                 'max_grad_norm': 1.0, 
                 'no_cuda': False, 
                 'model_type': 'koelectra-base-v3', 
                 'model_name_or_path': 'monologg/koelectra-base-v3-discriminator', 
                 'output_dir': '_checkpoints', 
                 'seed': 42, 
                 'train_batch_size': 32, 
                 'eval_batch_size': 32, 
                 'logging_steps': 250, 
                 'save_steps': 250, 
                 'learning_rate': 5e-05
     }
)
args.data_dir = os.path.join(args.data_dir, args.task)
args.output_dir = args.task + args.output_dir

In [ ]:
# initialize logging
import logging

logging.basicConfig(
    format= "%(asctime)s - %(message)s",
    # format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%H:%M:%S",
    level=logging.INFO,
)

In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter

# set seed
torch.cuda.manual_seed_all(args.seed)


#TODO: log versioning is not yet done
writer = SummaryWriter()

In [ ]:
# make Input Example
class InputExample(object):
    """
    A single training/test example for simple sequence classification.
    """

    def __init__(self, review_token, label):
        self.review_token = review_token
        self.label = label

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"
        
# make Input Feature
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, attention_mask, token_type_ids, label):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.token_type_ids = token_type_ids
        self.label = label

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"


In [ ]:
if args.task == 'sentiment':
    args.logging_steps, args.save_steps = 63, 63
    args.max_seq_len = 64
    args.train_batch_size = 128

from torch.utils.data import TensorDataset

def load_dataset(args, tokenizer, mode):
    features = get_features(args, tokenizer, mode)

    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor(
        [f.input_ids for f in features], 
        dtype=torch.long)
    all_attention_mask = torch.tensor(
        [f.attention_mask for f in features],
         dtype=torch.long)
    all_token_type_ids = torch.tensor(
        [f.token_type_ids for f in features],
        dtype=torch.long)

    if args.task == "classification":
        all_labels = torch.tensor([f.label for f in features],
                                  dtype=torch.long)
    else:
        all_labels = torch.tensor([f.label for f in features],
                                  dtype=torch.float)

    dataset = TensorDataset(all_input_ids,
                            all_attention_mask,
                            all_token_type_ids,
                            all_labels)
    return dataset


def get_features(args, tokenizer, mode):
    # load data and labels using Processor
    processor = Processor(args)
    try:
        examples = processor.get_examples(mode)
    except ValueError:
        print('possible modes: train, val, test')
    label_list = processor.get_labels()

    if args.task == 'classification':
      labels = [int(example.label) for example in examples]
    else:
      labels = [float(example.label) for example in examples]

    batch_encoding = tokenizer.batch_encode_plus(
        [(example.review_token) for example in examples],
        max_length=args.max_seq_len,
        padding="max_length",
        add_special_tokens=True,
        truncation=True,
    )

    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}
        
        # For xlm-roberta
        inputs["token_type_ids"] = [0] * len(inputs["input_ids"])  

        feature = InputFeatures(**inputs, label=labels[i])
        features.append(feature)

    return features


class Processor(object):
    def __init__(self, args):
        self.args = args

    def get_labels(self):
        if self.args.task == 'classification':
            return CLASSIFICATION_DICT.keys()
        else:
            return [None]

    def _read_file(cls, input_file):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8") as f:
            lines = []
            for line in f:
                lines.append(line.strip())
            return lines

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines[0:]):
            line = line.split("\t")
            review_token = line[0]
            label = line[1]
            examples.append(InputExample(review_token=review_token, label=label))
        return examples

    def get_examples(self, mode):
        """
        Args:
            mode: train, test
        """
        file_to_read = None
        if mode == "train":
            file_to_read = self.args.train_file
        elif mode == "test":
            file_to_read = self.args.test_file

        return self._create_examples(
            self._read_file(os.path.join(self.args.data_dir, file_to_read)), mode
        )

In [ ]:
config = args.config.from_pretrained(
            args.model_name_or_path,
            num_labels=17 if args.task == 'classification' else 1
        )
tokenizer = args.tokenizer.from_pretrained(
    args.model_name_or_path,
    do_lower_case=args.do_lower_case
)
model = args.model.from_pretrained(
    args.model_name_or_path,
    config=config
)

train_dataset = load_dataset(args, tokenizer, mode="train")
val_dataset = load_dataset(args, tokenizer, mode="test")
datasets = {'train': train_dataset, 'val': val_dataset}

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir='runs'

In [ ]:
import argparse
import json
import logging
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from transformers import (
    AdamW,
    get_linear_schedule_with_warmup
)
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from fastprogress.fastprogress import master_bar, progress_bar
import torch

# TODO: acc. calculating logic impl. + figure adding logic impl.
        
def train(args, model, datasets):
    train_sampler = RandomSampler(datasets['train'])
    train_loader = DataLoader(datasets['train'],
                              sampler=train_sampler,
                              batch_size=args.train_batch_size)
    val_sampler = RandomSampler(datasets['val'])
    val_loader = DataLoader(datasets['val'],
                            sampler=val_sampler,
                            batch_size=args.eval_batch_size)
    dataloaders = {'train': train_loader, 'val': val_loader}
    full_text = pd.read_csv(os.path.join(args.data_dir, 'full_text.txt'),
                            sep='\t')
    
    t_total = (
        len(dataloaders['train']) // args.gradient_accumulation_steps
        * args.num_train_epochs
    )
   
    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {
            'params': [p for n, p in model.named_parameters()
                    if not any(nd in n for nd in no_decay)],
            'weight_decay': args.weight_decay
         },
        {
            'params': [p for n, p in model.named_parameters()
                    if any(nd in n for nd in no_decay)],
            'weight_decay': 0.0
         }
    ]
    optimizer = AdamW(optimizer_grouped_parameters,
                      lr=args.learning_rate,
                      eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(t_total * args.warmup_proportion),
        num_training_steps=t_total
        )

    if os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt")) \
    and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt")):
        # Load optimizer and scheduler states
        optimizer.load_state_dict(
            torch.load(os.path.join(args.model_name_or_path, "optimizer.pt"))
            )
        scheduler.load_state_dict(
            torch.load(os.path.join(args.model_name_or_path, "scheduler.pt"))
            )

    print(f'training started [TASK: {args.task}]')
    global_step = 0
    tr_loss = 0.0

    model.zero_grad()
    mb = master_bar(range(int(args.num_train_epochs)))
    for epoch in mb:
        mb.main_bar.comment = f'training in progress: epoch {epoch+1}'
        accuracies, losses = {}, {}
        for stage, dataloader in dataloaders.items():
            epoch_iterator = progress_bar(dataloader, parent=mb)
            num_correct, num_items = 0, 0
            running_loss = 0
            for step, batch in enumerate(epoch_iterator):
                mb.child.comment = f'epoch {epoch+1} {stage}'
                if stage == 'train':
                    model.train()
                    optimizer.zero_grad() # why is this not in original func?
                else:
                    model.eval()
                
                batch = tuple(t.to(args.device) for t in batch)
                inputs = {
                    "input_ids": batch[0],
                    "attention_mask": batch[1],
                    "labels": batch[3]
                }
                inputs["token_type_ids"] = batch[2]  # we don't use this

                # model output: [loss, grad_fn (NLL), logits, batches]
                outputs = model(**inputs)
                if args.task == 'classification':
                    preds = torch.argmax(outputs.logits, dim=1)
                else:
                    preds = torch.round(outputs.logits).view(-1)
                num_items += preds.shape[0]
                num_correct += (inputs['labels'] == preds).sum()

                loss = outputs[0]

                if args.gradient_accumulation_steps > 1:
                    loss = loss / args.gradient_accumulation_steps

                if stage == 'train':
                    loss.backward()
                running_loss += loss.item()

                if (step + 1) % args.gradient_accumulation_steps == 0 \
                or (len(dataloader) <= args.gradient_accumulation_steps \
                    and (step + 1) == len(dataloader)) \
                and stage == 'train':
                    torch.nn.utils.clip_grad_norm_(model.parameters(),
                                                   args.max_grad_norm)

                    optimizer.step()
                    scheduler.step()
                    model.zero_grad()
                    global_step += 1

                    # TODO: evaluate function commented out for test exec
                    # if args.logging_steps > 0 \
                    # and global_step % args.logging_steps == 0:
                        # if args.evaluate_test_during_training:
                        #     evaluate(args, model, full_text, test_dataset,
                        #              "test",
                        #              global_step)

                    # if args.save_steps > 0 \
                    # and global_step % args.save_steps == 0:
                    #     # Save model checkpoint
                    #     output_dir = os.path.join(
                    #         args.output_dir,
                    #         "checkpoint-{}".format(global_step)
                    #         )
                    #     if not os.path.exists(output_dir):
                    #         os.makedirs(output_dir)
                    #     model_to_save = (
                    #         model.module if hasattr(model, "module") else model
                    #     )
                    #     model_to_save.save_pretrained(output_dir)

                    #     torch.save(args, os.path.join(output_dir,
                    #                                   "training_args.bin"))

                    #     if args.save_optimizer:
                    #         torch.save(optimizer.state_dict(),
                    #                    os.path.join(output_dir, "optimizer.pt"))
                    #         torch.save(scheduler.state_dict(),
                    #                    os.path.join(output_dir, "scheduler.pt"))
                if args.max_steps > 0 and global_step > args.max_steps:
                    break
            # if stage == 'val':
                # fig = plt.figure(figsize=(15, 8))
                # plt.subplot(2, 1, 1)
                # plt.title('Bar Chart of Labels Count and Accuracy', fontsize=15)
                # clrs = ['r' if (x < 15) else 'b' for x in count_list]
                # p1 = plt.bar(index, count_list,
                #             bar_width,
                #             color=clrs,
                #             alpha=alpha,
                #             label='Count')
                # plt.ylabel('Count of Labels', fontsize=12)
                # plt.xticks([], [])
                # plt.legend((p1[0],), ('Count',), fontsize=10)
                # plt.subplot(2, 1, 2)
                # p2 = plt.bar(index + bar_width, acc_tot,
                #             bar_width,
                #             color=clrs,
                #             alpha=alpha,
                #             label='Accuracy')
                # plt.gca().yaxis.set_major_formatter(mticker.PercentFormatter())
                # plt.ylabel('Accuracy by Labels', fontsize=12)
                # plt.xlabel('Label', fontsize=12)
                # plt.xticks(index, label_lst, fontsize=10, rotation=45)
                # plt.legend((p2[0],), ('Accuracy',), fontsize=10)
                # plt.show()
                    # for i in range(len(out_label_ids)):
                    #     print(tokenizer.decode(out_ids[i]), out_label_ids[i], preds[i])
                    # print(type(out_label_ids), type(preds))
                    # print(out_label_ids, preds)
                    #

                # log matplotlib to tensorboard
                # writer.add_figure('bar chart of labels count and accuracy', fig)
            accuracies[stage] = num_correct / num_items
            losses[stage] = running_loss / len(dataloader)
        writer.add_scalars(main_tag='accuracy',
                           tag_scalar_dict=accuracies,
                           global_step=epoch+1)
        writer.add_scalars(main_tag='loss',
                          tag_scalar_dict=losses,
                          global_step=epoch+1)
        if args.max_steps > 0 and global_step > args.max_steps:
            break

    return

In [ ]:
torch.cuda.empty_cache()
torch.cuda.ipc_collect()
torch.cuda.memory_summary(abbreviated=True)

In [ ]:
# GPU or CPU
args.device = "cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu"
model.to(args.device)

# Train
train(args, model, datasets)


In [ ]:
def evaluate(args, model, full_text, eval_dataset, mode, global_step=None):
    results = {}
    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

    # Eval!
    if global_step != None:
        logger.info("***** Running evaluation on {} dataset ({} step) *****".format(mode, global_step))
    else:
        logger.info("***** Running evaluation on {} dataset *****".format(mode))
    logger.info("  Num examples = {}".format(len(eval_dataset)))
    logger.info("  Eval Batch size = {}".format(args.eval_batch_size))
    eval_loss = 0.0
    # nb_eval_steps = 0
    preds = None
    preds_save = None
    out_label_ids = None
    out_input_ids = None

    for batch in progress_bar(eval_dataloader):
        preds_temp = None
        model.eval()
        batch = tuple(t.to(args.device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "labels": batch[3]
            }

            if args.model_type not in ["distilkobert", "xlm-roberta"]:
                inputs["token_type_ids"] = batch[2]  # Distilkobert, XLM-Roberta don't use segment_ids

            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]

            eval_loss += tmp_eval_loss.mean().item()
        # nb_eval_steps += 1
        if preds is None:
            out_input_ids = inputs['input_ids'].detach().cpu().numpy()
            preds = logits.detach().cpu().numpy()
            preds_save = preds.copy()
            # 12번째
            # if args.task == 'sentiment':
            #   for i in range(len(preds)):
            #     if preds[i][0] < 0.5:
            #       preds[i][0] = 0
            #     else:
            #       preds[i][0] = 1
            out_label_ids = inputs["labels"].detach().cpu().numpy()
        else:
            out_input_ids = np.append(out_input_ids, inputs['input_ids'].detach().cpu().numpy(), axis=0)
            preds_temp = logits.detach().cpu().numpy()
            # 12번째
            # if args.task == 'sentiment':
            #   for i in range(len(preds_temp)):
            #     if preds_temp[i][0] < 0.5:
            #       print('yes')
            #       preds_temp[i][0] = 0
            #     else:
            #       preds_temp[i][0] = 1
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            preds_save = np.append(preds_save, logits.detach().cpu().numpy(), axis=0)
            # preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)

        # 내가 수정한 부분
        
        if args.task == 'sentiment':
            preds[:][0] = torch.round(preds[:][0])

    tokenizer = args.tokenizer.from_pretrained(
        args.model_name_or_path,
        do_lower_case=args.do_lower_case
    )
    # label_dict = {'칭찬': 0, '중립': 0.5, '불만': 1}
    if args.task == 'classification':
        label_dict = {'중립': 0, '상담원': 1, '상담시스템': 2, '혜택': 3,
                      '할부금융상품': 4, '카드상품': 5, '청구입금': 6, '심사/한도': 7,
                      '생활편의서비스': 8, '상담/채널': 9, '리스렌탈상품': 10,
                      '라이프서비스': 11, '금융상품': 12,'고객정보관리': 13,
                      '가맹점매출/승인': 14, '삼성카드': 15, '기타': 16}
    else:
      label_dict = {'칭찬': 0, '불만': 1}
    label_dict = dict((v, k) for k, v in label_dict.items())
    df_review = []
    # temp_review = []
    df_label = np.vectorize(label_dict.get)(out_label_ids)
    df_prediction = np.vectorize(label_dict.get(np.argmax(preds)))
    for i in range(len(out_input_ids)):
        review_list = list(out_input_ids[i])


        # temp_review.append(str(x) for x in out_input_ids[i])


        while 0 in review_list:
            review_list.remove(0)
        del review_list[0]
        del review_list[-1]
        df_review.append(tokenizer.decode(review_list))
        # print(review_list, label_dict[out_label_ids[i] - 1], label_dict[np.argmax(preds[i]) - 1])
    df_data = {'Review': df_review, 'Label': df_label, 'Prediction': df_prediction}
    df = pd.DataFrame(df_data)

    df_train_data = {'Review': full_text['Review'], 'Label': full_text['Label']}
    df_from_train = pd.DataFrame(df_train_data)
    # Dodged Bar Chart (with same X coordinates side by side)

    bar_width = 0.35
    alpha = 0.5
    label_lst = list(label_dict.values())
    index = np.arange(len(label_lst))
    count_list, cnt_pred, cnt_label = [0 for _ in range(len(label_dict))], \
                                    [0 for _ in range(len(label_dict))], [0 for _ in range(len(label_dict))]
    count_labels = df_from_train.groupby('Label', as_index=False).Review.count()
    acc_labels = df[df['Label'] == df['Prediction']].groupby('Label').Review.count()
    viable_label = list(count_labels['Label'])
    for i in range(len(viable_label)):
        count_list[viable_label[i]] = count_labels['Review'][i]
    for validation in range(len(out_label_ids)):
        if args.task == 'sentiment':
          if int(out_label_ids[validation]) == int(preds[validation][0]):
              cnt_pred[int(out_label_ids[validation])] += 100
          cnt_label[int(out_label_ids[validation])] += 1
        else:
          if out_label_ids[validation] == np.argmax(preds, axis=1)[validation]:
              cnt_pred[int(out_label_ids[validation])] += 100
          cnt_label[int(out_label_ids[validation])] += 1
    for i in range(len(cnt_label)):
        if cnt_label[i] == 0:
            cnt_label[i] = 1
    acc_tot = np.divide(cnt_pred, cnt_label)
    acc_tot[np.isnan(acc_tot)] = 0
    print(count_list)
    print(acc_tot)
    fig = plt.figure(figsize=(15, 8))
    plt.subplot(2, 1, 1)
    plt.title('Bar Chart of Labels Count and Accuracy', fontsize=15)
    clrs = ['r' if (x < 15) else 'b' for x in count_list]
    p1 = plt.bar(index, count_list,
                 bar_width,
                 color=clrs,
                 alpha=alpha,
                 label='Count')
    plt.ylabel('Count of Labels', fontsize=12)
    plt.xticks([], [])
    # plt.legend((p1[0],), ('Count',), fontsize=10)
    plt.subplot(2, 1, 2)
    p2 = plt.bar(index + bar_width, acc_tot,
                 bar_width,
                 color=clrs,
                 alpha=alpha,
                 label='Accuracy')
    plt.gca().yaxis.set_major_formatter(mticker.PercentFormatter())
    plt.ylabel('Accuracy by Labels', fontsize=12)
    plt.xlabel('Label', fontsize=12)
    plt.xticks(index, label_lst, fontsize=10, rotation=45)
    # plt.legend((p2[0],), ('Accuracy',), fontsize=10)
    # plt.show()
        # for i in range(len(out_label_ids)):
        #     print(tokenizer.decode(out_ids[i]), out_label_ids[i], preds[i])
        # print(type(out_label_ids), type(preds))
        # print(out_label_ids, preds)
        #

    # log matplotlib to tensorboard
    writer.add_figure('bar chart of labels count and accuracy', fig)

    # eval_loss = eval_loss / nb_eval_steps
    # if output_modes[args.task] == "classification":


    preds_original = preds.copy()

    # 9번째
    # preds = np.argmax(preds, axis=1)
    if args.task == 'classification':
      preds = np.argmax(preds, axis=1)
    else:
      preds = np.squeeze(preds)


    # elif output_modes[args.task] == "regression":
    #     preds = np.squeeze(preds)
    result = compute_metrics(out_label_ids, preds)

    # 틀린 항목 정리
    check_right = out_label_ids == preds
    df = pd.DataFrame(columns=['Review', 'Label', 'Prediction', 'Softmax'])
    for check_result in range(len(check_right)):
        if not check_right[check_result]:
            review_list = list(out_input_ids[check_result])
            while 0 in review_list:
                review_list.remove(0)
            del review_list[0]
            del review_list[-1]
            df = df.append({'Review': tokenizer.decode(review_list), 
                           'Label': label_dict[out_label_ids[check_result]], 
                           'Prediction': label_dict[preds[check_result]],
                           'Softmax': preds_save[check_result]},
                           ignore_index=True
                           )
    
    print(df.head())
    df.to_csv(r'analysis_ckpt_{}_{}.csv'.format(args.task, global_step), header=None, index=None, sep='\t', mode='a')
            # print(tokenizer.decode(review_list), 'Label: ' + str(label_dict[out_label_ids[check_result]]) + ' Prediction: ' + str(
            #     label_dict[preds[check_result]]))


    # numpy_data = np.array(out_label_ids, preds)
    # df = pd.DataFrame(data=numpy_data, index=["row1", "row2"], columns=["column1", "column2"])
    print(result)
    results.update(result)
    output_dir = os.path.join(args.output_dir, mode)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    output_eval_file = os.path.join(output_dir,
                                    "{}-{}.txt".format(mode, global_step) if global_step else "{}.txt".format(mode))
    with open(output_eval_file, "w") as f_w:
        logger.info("***** Eval results on {} dataset *****".format(mode))
        for key in sorted(results.keys()):
            logger.info("  {} = {}".format(key, str(results[key])))
            f_w.write("  {} = {}\n".format(key, str(results[key])))

    return results

def compute_metrics(labels, preds):
    assert len(preds) == len(labels)

    # TODO: need to separate tr/val acc and loss + log all info in tensorboard
    result = {"acc": (labels == preds).mean()}
    writer.add_scalar(tag='val_acc',
                      scalar_value=result['acc'])
    
    return result

In [ ]:


if task == 'sentiment':
  args.logging_steps, args.save_steps = 63, 63
  args.max_seq_len = 64
  args.train_batch_size = 128
  args.learning_rate = 8e-06

processor = Processor(args)
labels = processor.get_labels()

config = args.config.from_pretrained(
            args.model_name_or_path,
            num_labels=len(labels)
        )
tokenizer = args.tokenizer.from_pretrained(
    args.model_name_or_path,
    do_lower_case=args.do_lower_case
)
model = args.model.from_pretrained(
    args.model_name_or_path,
    config=config
)

%cd /content/
train_dataset = load_and_cache_examples(args, tokenizer, mode="train")
val_dataset = load_and_cache_examples(args, tokenizer, mode='val')
test_dataset = load_and_cache_examples(args, tokenizer, mode="test")

In [ ]:
import torch
torch.cuda.empty_cache()
# torch.cuda.memory_summary(device=None, abbreviated=False)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir='runs'

In [ ]:
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from fastprogress.fastprogress import master_bar, progress_bar

# GPU or CPU
args.device = "cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu"
model.to(args.device)

# Train
global_step, tr_loss, full_text = train(args, model, train_dataset, test_dataset)
logger.info(" global_step = {}, average loss = {}".format(global_step, tr_loss))

In [ ]:
def test(args, model, test_data, mode, global_step=None):
  results = {}
  checkpoints = list(
              os.path.dirname(c) for c in
              sorted(glob.glob(args.output_dir + "/**/" + "pytorch_model.bin", recursive=True))
          )
  print(checkpoints)
  checkpoint = checkpoints[-1]
  logger.info("Test the following checkpoint: %s", checkpoint)
  global_step = checkpoint.split("-")[-1]
  model = args.model.from_pretrained(checkpoint)
  model.to(args.device)

  test_sampler = SequentialSampler(test_data)
  test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=args.eval_batch_size)

  # Eval!
  if global_step != None:
      logger.info("***** Running Test on {} dataset ({} step) *****".format(mode, global_step))
  else:
      logger.info("***** Running test on {} dataset *****".format(mode))
  logger.info("  Num examples = {}".format(len(test_data)))
  logger.info("  Eval Batch size = {}".format(args.eval_batch_size))
  eval_loss = 0.0
  # nb_eval_steps = 0
  preds = None
  preds_save = None
  out_input_ids = None

  for batch in progress_bar(test_dataloader):
      preds_temp = None
      model.eval()
      batch = tuple(t.to(args.device) for t in batch)

      with torch.no_grad():
          inputs = {
              "input_ids": batch[0],
              "attention_mask": batch[1],

              # label 없이 할 땐 여기 주석 반전
              "labels": batch[3]

          }

          # if args.model_type not in ["distilkobert", "xlm-roberta"]:
          inputs["token_type_ids"] = batch[2]  # Distilkobert, XLM-Roberta don't use segment_ids

          outputs = model(**inputs)

          # label 없이 할 땐 여기 주석 반전
          tmp_eval_loss, logits = outputs[:2]
          # logits = outputs[0]
          eval_loss += tmp_eval_loss.mean().item()

      # nb_eval_steps += 1
      if preds is None:
          out_input_ids = inputs['input_ids'].detach().cpu().numpy()
          preds = logits.detach().cpu().numpy()
          preds_save = preds.copy()

          # label 없이 할 땐 여기 주석 반전
          out_label_ids = inputs["labels"].detach().cpu().numpy()

      else:
          out_input_ids = np.append(out_input_ids, inputs['input_ids'].detach().cpu().numpy(), axis=0)
          preds_temp = logits.detach().cpu().numpy()
          preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
          preds_save = np.append(preds_save, logits.detach().cpu().numpy(), axis=0)

          # label 없이 할 땐 여기 주석 반전
          out_label_ids = np.append(out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)

  if args.task == 'sentiment':
    for i in range(len(preds)):
      if preds[i][0] < 0.5:
        preds[i][0] = 0
      else:
        preds[i][0] = 1
  preds_original = preds.copy()
  if args.task == 'classification':
    preds = np.argmax(preds, axis=1)
  else:
    preds = np.squeeze(preds)
  
  # label 없이 할 땐 return 전 까지 주석 반전
  result = compute_metrics(out_label_ids, preds)
  tokenizer = args.tokenizer.from_pretrained(
      args.model_name_or_path,
      do_lower_case=args.do_lower_case
  )
  # label_dict = {'칭찬': 0, '중립': 0.5, '불만': 1}
  if args.task == 'classification':
    label_dict = {'중립': 0, '상담원': 1, '상담시스템': 2, '혜택': 3, '할부금융상품': 4,
                '카드상품': 5, '청구입금': 6, '심사/한도': 7, '생활편의서비스': 8,
                '상담/채널': 9, '리스렌탈상품': 10, '라이프서비스': 11, '금융상품': 12,
                '고객정보관리': 13, '가맹점매출/승인': 14, '삼성카드': 15, '기타': 16}
  else:
    label_dict = {'칭찬': 0, '불만': 1}
  label_dict = dict((v, k) for k, v in label_dict.items())
  check_right = out_label_ids == preds
  df = pd.DataFrame(columns=['Review', 'Label', 'Prediction', 'Softmax'])
  for check_result in range(len(check_right)):
      if not check_right[check_result]:
          review_list = list(out_input_ids[check_result])
          while 0 in review_list:
              review_list.remove(0)
          del review_list[0]
          del review_list[-1]
          df = df.append({'Review': tokenizer.decode(review_list), 
                          'Label': label_dict[out_label_ids[check_result]], 
                          'Prediction': label_dict[preds[check_result]],
                          'Softmax': preds_save[check_result]},
                          ignore_index=True
                          )
  
  print(df.head())

  result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
  results.update(result)
  print('test accuracy :', result)
  return preds

In [ ]:
test_result = test(args, model, val_dataset, 'val')

In [ ]:
list_preds = []

In [ ]:
list_preds.append(test_result)


In [ ]:
print(len(list_preds[0]))

In [ ]:
def complete_result(list_preds):
  label_dict_classification = {'중립': 0, '상담원': 1, '상담시스템': 2, '혜택': 3, '할부금융상품': 4,
                '카드상품': 5, '청구입금': 6, '심사/한도': 7, '생활편의서비스': 8,
                '상담/채널': 9, '리스렌탈상품': 10, '라이프서비스': 11, '금융상품': 12,
                '고객정보관리': 13, '가맹점매출/승인': 14, '삼성카드': 15, '기타': 16}
  label_dict_sentiment = {'칭찬': 0, '불만': 1}
  label_dict_classification = dict((v, k) for k, v in label_dict_classification.items())
  label_dict_sentiment = dict((v, k) for k, v in label_dict_sentiment.items())
  for i in label_dict_classification:
    if i not in [0, 1, 2, 16]:
      label_dict_classification[i] = '>삼성카드>' + label_dict_classification[i]
    else:
      if i == (1 or 2):
        label_dict_classification[i] = '>고객서비스>' + label_dict_classification[i]
      if i == 16:
        label_dict_classification[i] = '>' + label_dict_classification[i]
  df = pd.DataFrame(columns = ['INT'])
  sent_result = np.vectorize(label_dict_sentiment.get)(list_preds[0])
  # class_result = np.vectorize(label_dict_classification.get)(list_preds[1])
  # result_arr = np.core.defchararray.add(sent_result, class_result)
  df['INT'] = sent_result
  print(df)

In [ ]:
complete_result(list_preds)

In [ ]:
# 수정금지: 타임스탬프
_model_build_end_time = printt("Model building: Start")
print(_model_build_end_time - _model_build_start_time)

#모델 분류

In [ ]:
# 수정금지: 타임스탬프
_test_start_time = printt("TEST: Start")

In [ ]:
#TODO: 해당 블럭에 테스트 수행을 위한 코드를 넣으세요. (시간측정 구간)
#분석 파일은 tsv 파일로 제공되며, 제공되는 학습데이터 파일과 동일한 레이아웃(단, 정답 분류 =공백)으로 제공됩니다.




In [ ]:
# 수정금지: 타임스탬프
_test_end_time = printt("Model building: Start")
print(_test_end_time - _test_start_time)

# 결과출력

In [ ]:
#TODO:해당 블럭에 테스트 결과를 파일로 저장하는 코드를 넣으세요. (시간측정 제외)
#저장 파일은tsv 파일로 제공되는 학습데이터 파일과 동일한 레이아웃(단, 정답 분류 = 테스트 결과 도출된 양식)으로 저장


